In [4]:
import psycopg2
from collections import defaultdict
from datetime import datetime, timedelta

# Define your database connection parameters
db_params = {
    'dbname': 'fixtransactions',
    'user': 'scalp',
    'password': 'QAtr@de442',
    'host': '10.7.8.59',
    'port': '5433',
}

# Enter the date for which you want to calculate the positions
date_str = '2023-03-28'
date = datetime.strptime(date_str, '%Y-%m-%d')
next_day = date + timedelta(days=1)

# Connect to the database
connection = psycopg2.connect(**db_params)
cursor = connection.cursor()

# Query the fixmsg table for the given day
query = f"""
    SELECT tag1, tag55, tag54, tag38
    FROM fixmsg
    WHERE tag52 >= '{date_str}' AND tag52 < '{next_day.strftime('%Y-%m-%d')}';
"""
cursor.execute(query)
rows = cursor.fetchall()

# Compute the positions
positions = defaultdict(lambda: defaultdict(int))
for tag1, symbol, side, quantity in rows:
    if side == 'BUY':
        positions[tag1][symbol] += quantity
    elif side == 'SELL':
        positions[tag1][symbol] -= quantity

# Print the positions
for tag1, symbols in positions.items():
    print(f"Trader {tag1}:")
    for symbol, quantity in symbols.items():
        print(f"  {symbol}: {quantity}")

# Close the database connection
cursor.close()
connection.close()


In [11]:
for key, value in positions.items():
    print(f"{key}: {value}")

In [22]:
import psycopg2
from collections import defaultdict
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

# Function to compute the positions for a given trader
def compute_positions(tag1):
    connection = psycopg2.connect(host="10.7.8.59", database="fixtransactions", user="scalp", password="QAtr@de442", port='5433')
    cursor = connection.cursor()

    query = f"""
        SELECT tag55, tag54, tag38
        FROM fixmsg
        WHERE tag52 >= '2023-03-28' AND tag52 < '2023-03-29' AND tag1 = '{tag1}';
    """
    cursor.execute(query)
    rows = cursor.fetchall()

    # Compute the positions
    positions = defaultdict(float)  # Use float as the value type
    for tag55, tag54, tag38 in rows:
        if tag54 == '1':
            positions[tag55] += float(tag38)
        elif tag54 == '2':
            positions[tag55] -= float(tag38)

    cursor.close()
    connection.close()

    return tag1, positions

# List of traders (tag1 values)
traders = ['5P525705', '31P532753']
# '5646353',
# '1P525501',
# '5647492',
# '12P52852',
# '14P52874',
# '3P528403',
# '8P528918',
# '24P531126',
# '15P52595',
# '34P534728',
# '20P529221',
# '3618588',
# '17P52867',
# '30P532652',
# '6P528806',
# '22P530624',
# '26P531527',
# '3616407',
# '7P526007',
# '23P530525',
# '19P529523',
# '25P531426',
# '4P525604',
# '32P534563',
# '33P534612',
# '16P52836',
# '2P525802',
# '27P532248']  # Update this list with actual trader values

# Execute the compute_positions function concurrently for all traders
with ThreadPoolExecutor() as executor:
    results = list(executor.map(compute_positions, traders))

# Print the positions
for tag1, positions in results:
    print(f"Trader {tag1}:")
    for tag55, tag38 in positions.items():
        print(f"  {tag55}: {tag38}")

Trader 5P525705:
  GOOGL: 569.0
  GOOG: 1485.0
  SPY: 363.0
  GDX: 15.0
  SCHW: -29.0
  RIOT: 15.0
  BITI: 415.0
  FSLR: 40.0
  NVDA: 22742.0
  DWAC: 28.0
  AFRM: -25.0
  AAPL: 254.0
  UPST: 0.0
  MSFT: 55.0
  SPXW: 26.0
  VXX: 34.0
  SPX: -88.0
  META: -11.0
  BITO: 17654.0
  BABA: -173.0
  PBR: 45.0
  MRK: 88.0
  FDX: 1767.0
  TSLA: -10.0
  JPM: 15.0
  F: -22.0
  AMZN: 76.0
  GS: 0.0
  ANET: -5.0
  GE: 0.0
  GLD: -35.0
  AMD: 32.0
  V: -20.0
  MCD: 0.0
  ATVI: -50.0
  GME: 0.0
  NFLX: -26.0
  DIA: -33.0
  FRC: -20.0
  MMM: -40.0
  T1: 27.0
  BXP: -76.0
  KRE: 25.0
Trader 31P532753:
  GOOG: 302.0
  GOOGL: 1815.0
  ABBV: -91.0
  MRNA: 23.0
  NVDA: -147.0
  BABA: 134.0
  BIDU: -10.0
  ZS: 0.0
  MS: -45.0
  SAVA: -8.0
  FUBO: -105.0
  MSFT: -76.0
  DDOG: 85.0
  RIOT: -5.0
  AMC: -15.0
  META: -30.0
  AAPL: -5.0
  GME: 0.0
  USO: -14.0
  LCID: -20.0
  LULU: -691.0
  AMGN: -25.0


In [33]:
import psycopg2
from collections import defaultdict
import datetime

def compute_positions(tag1):
    connection = psycopg2.connect(host="10.7.8.59", database="fixtransactions", user="scalp", password="QAtr@de442", port='5433')
    cursor = connection.cursor()

    query = f"""
    SELECT tag55, tag54, tag38, tag52, tag200, tag201, tag202, tag205
    FROM fixmsg
    WHERE tag52 >= '2023-03-28' AND tag52 < '2023-03-29'
    AND tag1 = '{tag1}'
    AND tag167 = 'OPT'
    AND tag39 = '2'
    AND tag200 IS NOT NULL
    AND tag201 IS NOT NULL
    AND tag202 IS NOT NULL
    AND tag205 IS NOT NULL;
    """
    cursor.execute(query)
    rows = cursor.fetchall()

    record_count = len(rows)
    print(f"Trader {tag1}: {record_count} records processed")

    positions_before_313 = defaultdict(float)
    positions_313_to_315 = defaultdict(float)

    for row in rows:
        tag55, side, quantity, timestamp, tag200, tag201, tag202, tag205 = row
        symbol = f"{tag55}{tag200}{tag205}{tag201}{tag202}"
        quantity = float(quantity) * (1 if side == '1' else -1)

        # Separate positions before 3:13 PM and positions between 3:13 PM and 3:15 PM
        if timestamp < datetime.datetime(2023, 3, 28, 15, 13):
            positions_before_313[symbol] += quantity
        elif datetime.datetime(2023, 3, 28, 15, 13) <= timestamp < datetime.datetime(2023, 3, 28, 15, 15):
            positions_313_to_315[symbol] += quantity

    cursor.close()
    connection.close()

    return positions_before_313, positions_313_to_315


traders = ['5P525705', '31P532753', '5646353', '1P525501', '5647492', '12P52852', '14P52874', '3P528403', '8P528918']

for tag1 in traders:
    positions_before_313, positions_313_to_315 = compute_positions(tag1)
    print(f"Trader {tag1}:")
    print("  Positions before 3:13 PM:")
    for symbol, quantity in positions_before_313.items():
        print(f"    {symbol}: {quantity}")
    print("  Positions between 3:13 PM and 3:15 PM:")
    for symbol, quantity in positions_313_to_315.items():
        print(f"    {symbol}: {quantity}")
    print()


Trader 5P525705: 28 records processed
Trader 5P525705:
  Positions before 3:13 PM:
    GOOG20241220055.00: 1.0
    GOOGL202409201185.00: -1.0
    FSLR20230550190.00: 0.0
    AAPL202307211150.00: -2.0
    AAPL202501171190.00: 2.0
  Positions between 3:13 PM and 3:15 PM:

Trader 31P532753: 0 records processed
Trader 31P532753:
  Positions before 3:13 PM:
  Positions between 3:13 PM and 3:15 PM:

Trader 5646353: 23 records processed
Trader 5646353:
  Positions before 3:13 PM:
  Positions between 3:13 PM and 3:15 PM:

Trader 1P525501: 0 records processed
Trader 1P525501:
  Positions before 3:13 PM:
  Positions between 3:13 PM and 3:15 PM:

Trader 5647492: 0 records processed
Trader 5647492:
  Positions before 3:13 PM:
  Positions between 3:13 PM and 3:15 PM:

Trader 12P52852: 0 records processed
Trader 12P52852:
  Positions before 3:13 PM:
  Positions between 3:13 PM and 3:15 PM:

Trader 14P52874: 24 records processed
Trader 14P52874:
  Positions before 3:13 PM:
    GOOGL202406211190.00: 9

In [15]:
import psycopg2
from collections import defaultdict
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

def compute_positions_and_orders(tag1, date):
    connection = psycopg2.connect(host="10.7.8.59", database="fixtransactions", user="scalp", password="QAtr@de442", port='5433')
    cursor = connection.cursor()

    query = f"""
    SELECT tag55, tag54, tag38, tag52, tag200, tag201, tag202, tag205, tag31, tag39
    FROM fixmsg
    WHERE tag52 >= timestamp '{date}' 
    AND tag52 < timestamp '{date}' + INTERVAL '1 DAY'
    AND tag1 = '{tag1}'
    AND tag167 = 'OPT'
    AND tag200 IS NOT NULL
    AND tag201 IS NOT NULL
    AND tag202 IS NOT NULL
    AND tag205 IS NOT NULL;
    """
    cursor.execute(query)
    rows = cursor.fetchall()

    record_count = len(rows)

    positions_before_313 = defaultdict(float)
    orders_313_to_315 = []

    closing_time = datetime.strptime(f'{date} 16:15:00', '%Y-%m-%d %H:%M:%S')
    time_window_start = closing_time - timedelta(minutes=6)

    for tag55, tag54, tag38, tag52, tag200, tag201, tag202, tag205, tag31, tag39 in rows:
        option_type = 'P' if tag201 == '0' else 'C'
        expiration_date = datetime.strptime(f"{tag200}{tag205}", '%Y%m%d').strftime('%Y%m%d')
        strike_price = f"{float(tag202):.0f}"
        option_contract_name = f"{tag55}{expiration_date}{option_type}{strike_price}"

        if tag52 < time_window_start:
            if tag39 == '2':
                positions_before_313[option_contract_name] += float(tag38) * (1 if tag54 == '1' else -1)
        elif time_window_start <= tag52 < closing_time:
            if option_contract_name in positions_before_313 and tag39 != '2':
                open_position = positions_before_313[option_contract_name] * (1 if tag54 == '1' else -1)
                if open_position != 0:
                    orders_313_to_315.append((option_contract_name, tag54, tag38, tag52))

    cursor.close()
    connection.close()

    return tag1, positions_before_313, orders_313_to_315, record_count


def run(date):
    traders = ['5P525705', 
    '31P532753',
    '5646353',
    '1P525501',
    '5647492',
    '12P52852',
    '14P52874',
    '3P528403',
    '8P528918',
    '24P531126',
    '15P52595',
    '34P534728',
    '20P529221',
    '3618588',
    '17P52867',
    '30P532652',
    '6P528806',
    '22P530624',
    '26P531527',
    '3616407',
    '7P526007',
    '23P530525',
    '19P529523',
    '25P531426',
    '4P525604',
    '32P534563',
    '33P534612',
    '16P52836',
    '2P525802',
    '27P532248']

    with ThreadPoolExecutor() as executor:
        results = list(executor.map(compute_positions_and_orders, traders, [date]*len(traders)))

    unfilled_orders = []

    for result in results:
        print("")
        tag1, positions_before_313, orders_313_to_315, record_count = result
        print(f"Trader {tag1}: {record_count} records processed")
        print("  Positions before 3:13 PM EST:")
        for option_contract_name, tag38 in positions_before_313.items():
            if tag38 != 0:
                print(f"    {option_contract_name}: {tag38}")
        unfilled_orders += [(tag1, *order) for order in orders_313_to_315]

    unfilled_orders = sorted(unfilled_orders, key=lambda x: x[3], reverse=True)
    print()
    print("╔══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗")
    print()
    print("Unfilled Orders between 3:13 PM and 3:15 PM EST:")
    for order in unfilled_orders:
        tag1, option_contract_name, tag54, tag38, tag52 = order
        action = "To Buy" if tag54 == "1" else "To Sell"
        if tag38 != 0:
            print(f"  Trader {tag1}: {action} {tag38} of {option_contract_name} at {tag52}")

run('2023-06-05')


Trader 5P525705: 3483 records processed
  Positions before 3:13 PM EST:

Trader 31P532753: 0 records processed
  Positions before 3:13 PM EST:

Trader 5646353: 344 records processed
  Positions before 3:13 PM EST:
    SPXW20230605C4250: 3.0

Trader 1P525501: 289 records processed
  Positions before 3:13 PM EST:

Trader 5647492: 14 records processed
  Positions before 3:13 PM EST:

Trader 12P52852: 998 records processed
  Positions before 3:13 PM EST:

Trader 14P52874: 0 records processed
  Positions before 3:13 PM EST:

Trader 3P528403: 2308 records processed
  Positions before 3:13 PM EST:

Trader 8P528918: 0 records processed
  Positions before 3:13 PM EST:

Trader 24P531126: 161 records processed
  Positions before 3:13 PM EST:
    HUM20230623C518: 1.0
    HUM20230609C512: -1.0
    ABNB20230609C113: 38.0
    MANU20230609C21: 10.0
    MANU20230609C18: 10.0

Trader 15P52595: 0 records processed
  Positions before 3:13 PM EST:

Trader 34P534728: 10935 records processed
  Positions bef